<h1> IMPORT LIBRARY <h1>

In [1]:
import pandas as pd
import os
import numpy as np
import time
import openpyxl
import glob

from datetime import datetime
from bs4 import BeautifulSoup
from io import StringIO

<h1> DEFINE AND CREATE FOLDERS <h1>

In [2]:
#folder paths
script_dir = os.getcwd()

folder_merge = r'\_MERGE'
merge_path = script_dir+folder_merge
if not os.path.exists(merge_path):
    # Create the directory
    os.makedirs(merge_path)

folder_final = r'\_FINAL'
final_path = script_dir+folder_final
if not os.path.exists(final_path):
    # Create the directory
    os.makedirs(final_path)

folder_qris = r'\QRIS'
qris_path = script_dir+folder_qris

folder_shopee = r'\SHOPEE'
shopee_path = script_dir+folder_shopee

folder_grab = r'\GRAB'
grab_path = script_dir+folder_grab

folder_gojek1 = r'\GO_Laporan'
gojek1_path = script_dir+folder_gojek1

folder_gojek2 = r'\GO_Mie'
gojek2_path = script_dir+folder_gojek2

folder_web = r'\WEB'
web_path = script_dir+folder_web

-----------
<h1>PREPARING GRAB<h1>

- Combining Grab Invoice Files

In [223]:
all_files = os.listdir(grab_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(grab_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path, low_memory=False)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_grab = pd.concat(df_list, ignore_index=True)

# Save the final result to a new CSV file
new_file_path = merge_path+"/"+'merged_GRAB.csv'

if os.path.exists(new_file_path):
    os.remove(new_file_path)

big_grab.to_csv(new_file_path, index=False)

- Cleaning Combined Grab File

In [251]:
merged_grab = pd.read_csv(merge_path+'\merged_grab.csv',sep=',',decimal='.')

reduced_grab   =   merged_grab.loc[:,["Store Name",
                                      "Updated On",
                                      "Status",
                                      "Short Order ID",
                                      "Net Sales",
                                      "Amount"]].rename(columns={ "Store Name"      :"CAB",
                                                                  "Updated On"      :"DATETIME",
                                                                  "Status"          :"Status",
                                                                  "Short Order ID"  :"ID",
                                                                  "Net Sales"       :"NOM",
                                                                  "Amount":"NOM2"}).fillna("")


reduced_grab['DATETIME']    =   pd.to_datetime(reduced_grab['DATETIME'], format='%d %b %Y %I:%M %p')
reduced_grab['DATE']        =   reduced_grab['DATETIME'].dt.strftime('%d/%m/%Y')
reduced_grab['TIME']        =   reduced_grab['DATETIME'].dt.time

reduced_grab                =  reduced_grab.drop(reduced_grab[reduced_grab['Status'] == 'Cancelled'].index)

reduced_grab['CODE']        =   ""
reduced_grab['KAT']         =   "GRAB FOOD"
reduced_grab['SOURCE']      =   "INVOICE"

reduced_grab.loc[reduced_grab['NOM'] == '', 'CODE'] = 'Adjustment'
reduced_grab.loc[reduced_grab['NOM'] == '', 'NOM'] = reduced_grab['NOM2']
reduced_grab['NOM']         =   pd.to_numeric(reduced_grab['NOM']).astype(int)

# re-order columns
reduced_grab  =   reduced_grab[["CAB", "DATE", "TIME", "CODE", "ID", "NOM", "KAT", "SOURCE"]]

# Save the final result to a new CSV file
new_file_path = final_path+"/"+'reduced_GRAB.csv'

# Save the final result to a new CSV file
reduced_grab.to_csv(os.path.join(final_path, 'reduced_GRAB.csv'), index=False)

-----------
<h1> PREPARING SHOPEE <h1>

- Combining Shopeefood Invoice Files

In [225]:
#combine Shopee
all_files = os.listdir(shopee_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(shopee_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_shopee = pd.concat(df_list, ignore_index=True)

# Save the final result to a new CSV file
new_file_path = merge_path+"/"+'merged_SHOPEE.csv'

if os.path.exists(new_file_path):
    os.remove(new_file_path)

big_shopee.to_csv(new_file_path, index=False)

- Cleaning Combined Shopeefood File

In [226]:
merged_shopee = pd.read_csv(merge_path+'\merged_SHOPEE.csv',sep=',',decimal='.')

reduced_shopee   =   merged_shopee.loc[:,["Order Pick up ID",
                                        "Store Name",
                                        "Order Complete/Cancel Time",
                                        "Order Amount",
                                        "Order Status"]].rename(columns={   "Order Pick up ID"          : "ID",
                                                                            "Store Name"                : "CAB",
                                                                            "Order Complete/Cancel Time": "DATETIME",
                                                                            "Order Amount"              : "NOM",
                                                                            "Order Status"              : "Status"}).fillna("")

reduced_shopee['DATETIME']    =   pd.to_datetime(reduced_shopee['DATETIME'], format='%d/%m/%Y %H:%M:%S')
reduced_shopee['DATE']        =   reduced_shopee['DATETIME'].dt.strftime('%d/%m/%Y')
reduced_shopee['TIME']        =   reduced_shopee['DATETIME'].dt.time

reduced_shopee['NOM']         =   pd.to_numeric(reduced_shopee['NOM']).astype(int)
reduced_shopee                =  reduced_shopee.drop(reduced_shopee[reduced_shopee['Status'] == 'Cancelled'].index)

reduced_shopee['CODE']        =   ""
reduced_shopee['KAT']         =   "SHOPEEPAY"
reduced_shopee['SOURCE']      =   "INVOICE"

# re-order columns
reduced_shopee                =   reduced_shopee[["CAB", "DATE", "TIME", "CODE", "ID", "NOM", "KAT", "SOURCE"]]

# Save the final result to a new CSV file
reduced_shopee.to_csv(os.path.join(final_path, 'reduced_SHOPEE.csv'), index=False)

-----------
<h1>PREPARING QRIS SHOPEE<h1>

- Combining QRIS Shopee Invoice Files

In [227]:
#combine QRIS Shopee
all_files = os.listdir(qris_path)

#Copy File to a new File
csv_files = [f for f in all_files if f.endswith('.csv') and not f.startswith('new ')]
for csv in csv_files:
    file_path = os.path.join(qris_path, csv)
    try:
        # Try reading the file using semicolon delimiter
        df = pd.read_csv(file_path, sep=';', escapechar='"')
        df.to_csv(os.path.join(qris_path, 'new ' + csv), sep=',', index=False)
        # df_new = pd.read_csv(qris_path + '\new' + csv)
        # df_list.append(df_new)
    except Exception as e:
        print(f"Could not read file {csv} and create its csv because: {e}")

# Combine New QRIS Files
new_files = glob.glob(os.path.join(qris_path, 'new' + '*.csv'))

df_list = []
for new in new_files:
    # file_path = glob.glob(os.path.join(qris_path, 'new' + '*.csv'))
    try:
        df = pd.read_csv(new)
        df_list.append(df)
    except Exception as e:
        print(f"Could not read file {new} and combine it because {e}")

# Concatenate all data with same separation into one dataframe
big_qris = pd.concat(df_list, ignore_index=True)
big_qris = big_qris.dropna(axis=1, how='all')

# Save the final result to a new CSV file
new_file_path = merge_path+"/"+'merged_QRIS.csv'

if os.path.exists(new_file_path):
    os.remove(new_file_path)
big_qris.to_csv(new_file_path, index=False)

- Cleaning Combined QRIS Shopee File

In [228]:
merged_qris = pd.read_csv(merge_path+'\merged_QRIS.csv', quotechar=None, delimiter=',', quoting=3, escapechar='"', decimal='.')
merged_qris.replace(0, np.nan, inplace=True)
merged_qris = merged_qris.dropna(axis=1, how='all')

#Fix Date/Time with Update Time
merged_qris.loc[merged_qris['Merchant Scope'] == '1', 'Create Time'] = np.nan
merged_qris['Create Time'] = merged_qris['Create Time'].astype('datetime64[ns]')
merged_qris['Terminal ID'] = merged_qris['Terminal ID'].astype('datetime64[ns]')
merged_qris.loc[merged_qris['Merchant Scope'] == '1', 'Create Time'] = merged_qris['Terminal ID']

#Fix Transaction Amount
merged_qris['Issuer Identifier'] = merged_qris['Issuer Identifier'].astype(object, errors='ignore')
merged_qris.loc[merged_qris['Merchant Scope'] == '1', 'Issuer Identifier'] = merged_qris['External Reference ID']
merged_qris['Issuer Identifier'] = merged_qris['Issuer Identifier'].astype('Float64')
merged_qris['Issuer Identifier'] = merged_qris['Issuer Identifier'].astype('int64')

#Fix Transaction ID
merged_qris.loc[merged_qris['Merchant Scope'] == '1', 'Merchant Scope'] = '1' + merged_qris['Transaction ID'].str[:-2] 

reduced_qris    =   merged_qris.loc[:,["Partner Merchant ID",
                                    "Create Time",
                                    "Merchant Scope",
                                    "Issuer Identifier",
                                    "Merchant/Store Name"]].rename(columns={ "Partner Merchant ID"  : "CAB",
                                                                            "Create Time"           : "DATETIME",
                                                                            "Merchant Scope"        : "ID",
                                                                            "Issuer Identifier"     : "NOM",
                                                                            "Merchant/Store Name"   : "Status"}).fillna("")

reduced_qris                =  reduced_qris.drop(reduced_qris[reduced_qris['CAB'] == ''].index)
reduced_qris                =  reduced_qris.drop(reduced_qris[reduced_qris['Status'] == 'Withdrawal'].index)

reduced_qris['DATETIME']    =   pd.to_datetime(reduced_qris['DATETIME'], format='%Y-%m-%d %H:%M:%S')
reduced_qris['DATE']        =   reduced_qris['DATETIME'].dt.strftime('%d/%m/%Y')
reduced_qris['TIME']        =   reduced_qris['DATETIME'].dt.time

reduced_qris['CODE']        =   ""
reduced_qris['KAT']         =   "QRIS SHOPEE"
reduced_qris['SOURCE']      =   "INVOICE"

# re-order columns
reduced_qris                =   reduced_qris[["CAB", "DATE", "TIME", "CODE", "ID", "NOM", "KAT", "SOURCE"]]

# Save the final result to a new CSV file
reduced_qris.to_csv(os.path.join(final_path, 'reduced_QRIS.csv'), index=False)

-----------
<h1>PREPARING GOJEK 1 <i>(Laporan Transaksi GoFood)<i><h1>

- Combining Gojek Invoice Files

In [229]:
#combine Gojek1
all_files = os.listdir(gojek1_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(gojek1_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df['Branch'] = csv.split(".")[0]
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df['Branch'] = csv
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Save the final result to a new CSV file
new_file_path = merge_path+"/"+'merged_GOJEK1.csv'

try:
    big_gojek1 = pd.concat(df_list, ignore_index=True)
    if os.path.exists(new_file_path):
        os.remove(new_file_path)
        big_gojek1.to_csv(new_file_path, index=False)
except:
    pass

- Cleaning Combined Gojek "<i>Laporan Transaksi...<i>" File

In [230]:
merged_gojek1 = pd.read_csv(merge_path+'\merged_GOJEK1.csv',sep=',',decimal='.')

reduced_gojek1   =   merged_gojek1.loc[:,["Waktu Selesai (WIB)",
                                        "Nomor Pesanan",
                                        "Harga Makanan",
                                        "Branch"]].rename(columns={ "Waktu Selesai (WIB)"   : "DATETIME",
                                                                    "Nomor Pesanan"         : "ID",
                                                                    "Harga Makanan"         : "NOM",
                                                                    "Branch"                : "CAB"}).fillna("")

reduced_gojek1['DATETIME']    =   pd.to_datetime(reduced_gojek1['DATETIME'])
reduced_gojek1['DATE']        =   reduced_gojek1['DATETIME'] .dt.strftime('%d/%m/%Y')
reduced_gojek1['TIME']        =   reduced_gojek1['DATETIME'] .dt.time

reduced_gojek1['NOM']         =   pd.to_numeric(reduced_gojek1['NOM']).astype(int)

reduced_gojek1['CODE']        =   ""
reduced_gojek1['KAT']         =   "GO RESTO"
reduced_gojek1['SOURCE']      =   "INVOICE"

# re-order columns
reduced_gojek1                =   reduced_gojek1[["CAB", "DATE", "TIME", "CODE", "ID", "NOM", "KAT", "SOURCE"]]

# Save the final result to a new CSV file
reduced_gojek1.to_csv(os.path.join(final_path, 'reduced_GOJEK1.csv'), index=False)

-----------
<h1>PREPARING GOJEK 2 <i>(Mie_Gacoan_..._P...)<i><h1>

- Combining

In [14]:
#combine Gojek2
all_files = os.listdir(gojek2_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(gojek2_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df['Branch'] = csv.split(".")[0]
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df['Branch'] = csv
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

new_file_path = merge_path+"/"+'merged_GOJEK2.csv'

try:
    big_gojek2 = pd.concat(df_list, ignore_index=True)
    if os.path.exists(new_file_path):
        os.remove(new_file_path)
        big_gojek2.to_csv(new_file_path, index=False)
except:
    pass

- Cleaning

In [255]:
merged_gojek2 = pd.read_csv(merge_path+'\merged_GOJEK2.csv',sep=',',decimal='.')
merged_gojek2 = merged_gojek2.dropna(axis=1, how='all')

reduced_gojek2   =   merged_gojek2.loc[:,["Transaction Time",
                                          "Order",
                                          "Amount",
                                          "Branch"]].rename(columns={ "Transaction Time": "DATETIME",
                                                                      "Order"           : "ID",
                                                                      "Amount"          : "NOM",
                                                                      "Branch"          : "CAB"}).fillna("")

reduced_gojek2['DATETIME']    =   pd.to_datetime(reduced_gojek2['DATETIME'], format='%b %d %Y, %H:%M')
reduced_gojek2['DATE']        =   reduced_gojek2['DATETIME'] .dt.strftime('%d/%m/%Y')
reduced_gojek2['TIME']        =   reduced_gojek2['DATETIME'] .dt.time

reduced_gojek2['NOM']         =   pd.to_numeric(reduced_gojek2['NOM']).astype(int)

reduced_gojek2['CODE']        =   ""
reduced_gojek2['KAT']         =   "GO RESTO"
reduced_gojek2['SOURCE']      =   "INVOICE"

# re-order columns
reduced_gojek2                =   reduced_gojek2[["CAB", "DATE", "TIME", "CODE", "ID", "NOM", "KAT", "SOURCE"]]

# Save the final result to a new CSV file
reduced_gojek2.to_csv(os.path.join(final_path, 'reduced_GOJEK2.csv'), index=False)


-----------
<h1>PREPARING WEB<h1>

- Converting html-styled .xls to .xlsx

In [233]:
xls_files = os.listdir(web_path)

#convert html-styled xls to xlsx
for file in xls_files:
    if file.endswith(".xls"):
        file_path = os.path.join(web_path, file)
        
        with open(file_path) as f:
            soup = BeautifulSoup(f, 'html.parser')

        tables = soup.find_all('table')
        writer = pd.ExcelWriter(file_path.replace(".xls", ".xlsx"), engine='openpyxl', mode='w')
        
        for i, table in enumerate(tables):
            caption = table.find('caption')
            if caption:
                sheet_name = caption.get_text().strip()
            else:
                sheet_name = 'Sheet{}'.format(i+1)
            df = pd.read_html(StringIO(str(table)))[0]
            df.drop(1, inplace=True)
            df.to_excel(sheet_name=sheet_name, index=False, header=False, excel_writer=writer,)
            writer.close()

- Converting .xlsx to .csv

In [234]:
# select xlsx files only
xlsx_files = glob.glob(os.path.join(web_path, '*.xlsx'))

#convert xlsx to csv
for xlsx in xlsx_files:
    try:
        read_xlsx = pd.read_excel(xlsx)
        file_name = xlsx[xlsx.find('WEB') + 4:-5]
        read_xlsx.to_csv(os.path.join(web_path, file_name + '.csv'), index = None, header=True)
    except Exception as e:
        print(f"error: {e}")
    

- Combining all .csv of WEB files

In [235]:
#combine WEB
all_files = os.listdir(web_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(web_path, csv)
    try:
        df = pd.read_csv(file_path)
        df_list.append(df)
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_web = pd.concat(df_list, ignore_index=True)

# Save the final result to a new CSV file
new_file_path = merge_path+"/"+'merged_WEB.csv'

if os.path.exists(new_file_path):
    os.remove(new_file_path)

big_web.to_csv(new_file_path, index=False)

- Cleaning combined web file

In [236]:
merged_web = pd.read_csv(merge_path+'\merged_web.csv',sep=',',decimal='.')

reduced_web                =  merged_web.loc[:,["DATE",
                                                "CAB",
                                                "CODE",
                                                "TIME.1",
                                                "KATEGORI",
                                                "CUSTOMER",
                                                "TOTAL"]].rename(columns={"TIME.1"      : "TIME",
                                                                        "KATEGORI"    : "KAT",
                                                                        "CUSTOMER"    : "ID",
                                                                        "TOTAL"       : "NOM"}).fillna("")

reduced_web                =  reduced_web.drop(reduced_web[reduced_web['DATE'] == 'TOTAL'].index)

reduced_web['DATE']        =   pd.to_datetime(reduced_web['DATE'], format='%Y-%m-%d')
reduced_web['DATE']        =   reduced_web['DATE'].dt.strftime('%d/%m/%Y')

reduced_web['NOM']         =   pd.to_numeric(reduced_web['NOM']).astype(int)

reduced_web['SOURCE']      =   "WEB"

# re-order columns
reduced_web                =   reduced_web[["CAB", "DATE", "TIME", "CODE", "ID", "NOM", "KAT", "SOURCE"]]

# Save the final result to a new CSV file
reduced_web.to_csv(os.path.join(final_path, 'reduced_web.csv'), index=False)


------------------
<h1>COMBINE ALL FINAL FILES <h1>

In [241]:
#combine ALL
all_files = os.listdir(final_path)

# Filter out non-CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list to hold the dataframes
df_list = []

for csv in csv_files:
    file_path = os.path.join(final_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        df_list.append(df)
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
big_ready = pd.concat(df_list, ignore_index=True)

# Save the final result to a new CSV file
new_file_path = script_dir+"/"+'Merge-Ready.xlsx'

if os.path.exists(new_file_path):
    os.remove(new_file_path)

big_ready.to_excel(new_file_path, index=False)